# UHF SCF

In [1]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

np.set_printoptions(precision=3, linewidth=120)

## Molecular setup

UHF SCF calculations start off in the same way as RHF SCF calculations. We first need a second-quantized Hamiltonian that expresses all the integrals in the AO basis.

In [2]:
molecule = gqcpy.Molecule.HRingFromDistance(3, 1.0, 0)  # create a neutral molecule
N = molecule.numberOfElectrons()
N_alpha = N//2 + 1
N_beta = N//2

spinor_basis = gqcpy.RSpinOrbitalBasis(molecule, "STO-3G")
S = spinor_basis.quantizeOverlapOperator().parameters()

sq_hamiltonian = gqcpy.SQHamiltonian.Molecular(spinor_basis, molecule)  # 'sq' for 'second-quantized'

In [3]:
print(molecule)

Number of electrons: 3
H  (0.57735, 0, 0)
H  (-0.288675, 0.5, 0)
H  (-0.288675, -0.5, 0)



## Plain UHF SCF

Then, we need an UHF SCF solver and an UHF SCF environment.

In [4]:
environment = gqcpy.UHFSCFEnvironment.WithCoreGuess(N_alpha, N_beta, sq_hamiltonian, S)
plain_solver = gqcpy.UHFSCFSolver.Plain(threshold=1.0e-04, maximum_number_of_iterations=1000)  # the system is not converging very rapidly

Using GQCP's capabilities of 'injecting' Python code inside the C++ library, we'll add an intermediary step that doesn't modify the environment, but just prints out the current Fock matrices.

This is done by writing a `Python` function that manipulates the solver's environment (of type `EnvironmentType`), and subsequently wrapping that function into a `FunctionalStep_EnvironmentType`. The instance of type `FunctionalStep_EnvironmentType` is then subsequently inserted inside the algorithm that should be performed.

In this example, we'll print out the alpha- and beta- Fock matrices.

In [5]:
def print_fock_matrices(environment):
    if (environment.fock_matrices_alpha):
        print("Alpha Fock matrix:\n{}\n".format(environment.fock_matrices_alpha[-1]))
        print("Beta Fock matrix:\n{}".format(environment.fock_matrices_beta[-1]))
    else:
        print("No Fock matrices in the queue.")

print_step = gqcpy.FunctionalStep_UHFSCFEnvironment(print_fock_matrices)

In [6]:
plain_solver.insert(print_step, index=0)  # insert at the front

We can use the solver to optimize the UHF parameters.

In [7]:
qc_structure = gqcpy.UHF.optimize(plain_solver, environment)

56 -0.941]
 [-0.94  -0.941 -0.77 ]]

Beta Fock matrix:
[[-0.604 -0.795 -0.837]
 [-0.795 -0.605 -0.837]
 [-0.837 -0.837 -0.727]]
Alpha Fock matrix:
[[-0.857 -0.895 -0.94 ]
 [-0.895 -0.856 -0.941]
 [-0.94  -0.941 -0.77 ]]

Beta Fock matrix:
[[-0.604 -0.795 -0.837]
 [-0.795 -0.605 -0.837]
 [-0.837 -0.837 -0.727]]
Alpha Fock matrix:
[[-0.857 -0.895 -0.94 ]
 [-0.895 -0.856 -0.941]
 [-0.94  -0.941 -0.77 ]]

Beta Fock matrix:
[[-0.604 -0.795 -0.837]
 [-0.795 -0.605 -0.837]
 [-0.837 -0.837 -0.727]]
Alpha Fock matrix:
[[-0.857 -0.895 -0.94 ]
 [-0.895 -0.856 -0.941]
 [-0.94  -0.941 -0.77 ]]

Beta Fock matrix:
[[-0.604 -0.795 -0.837]
 [-0.795 -0.605 -0.837]
 [-0.837 -0.837 -0.727]]
Alpha Fock matrix:
[[-0.857 -0.895 -0.94 ]
 [-0.895 -0.856 -0.941]
 [-0.94  -0.941 -0.77 ]]

Beta Fock matrix:
[[-0.604 -0.795 -0.837]
 [-0.795 -0.605 -0.837]
 [-0.837 -0.837 -0.727]]
Alpha Fock matrix:
[[-0.857 -0.895 -0.94 ]
 [-0.895 -0.856 -0.941]
 [-0.94  -0.941 -0.77 ]]

Beta Fock matrix:
[[-0.604 -0.795 -0.837]
 

In [8]:
print(qc_structure.groundStateEnergy() + gqcpy.Operator.NuclearRepulsion(molecule).value())

-0.6311463074240042


In [9]:
uhf_parameters = qc_structure.groundStateParameters()

In [10]:
print(uhf_parameters.orbitalEnergies(gqcpy.Spin.alpha))

[-1.033  0.187  0.769]


In [11]:
print(uhf_parameters.orbitalEnergies(gqcpy.Spin.beta))

[-0.89   0.821  0.937]


In [12]:
print(plain_solver.numberOfIterations())

649


## DIIS UHF SCF

In [13]:
environment = gqcpy.UHFSCFEnvironment.WithCoreGuess(N_alpha, N_beta, sq_hamiltonian, S)
diis_solver = gqcpy.UHFSCFSolver.DIIS(threshold=1.0e-04, maximum_number_of_iterations=1000, minimum_subspace_dimension=6, maximum_subspace_dimension=6)  # the system is not converging very rapidly

In [14]:
qc_structure = gqcpy.UHF.optimize(diis_solver, environment)

In [15]:
print(qc_structure.groundStateEnergy() + gqcpy.Operator.NuclearRepulsion(molecule).value())

-0.6311235468342336


In [16]:
uhf_parameters = qc_structure.groundStateParameters()

In [17]:
print(uhf_parameters.orbitalEnergies(gqcpy.Spin.alpha))

[-1.033  0.187  0.769]


In [18]:
print(uhf_parameters.orbitalEnergies(gqcpy.Spin.beta))

[-0.89   0.821  0.937]


In [19]:
print(diis_solver.numberOfIterations())

163
